In [1]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import cvxopt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def select_train(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

def select_test(list_var, data):
    for var in list_var :
        data.pop(var)
    return data

In [3]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()

# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset1 = raw_dataset.copy()
test_dataset1.isna().sum()
test_dataset1 = test_dataset1.dropna()

# 2/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim2.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset2 = raw_dataset.copy()
test_dataset2.isna().sum()
test_dataset2 = test_dataset2.dropna()

# 3/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv','nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim3.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset3 = raw_dataset.copy()
test_dataset3.isna().sum()
test_dataset3 = test_dataset3.dropna()

In [4]:
list_comp = ['input_width', 'input_size','nb_params_conv', 'nb_params_bn', 'nb_params_fc', 'nb_layers', 
                'nb_conv_layers', 'nb_bn_layers','nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']

list_var = ['input_width', 'nb_params_bn', 'nb_params_fc','nb_conv_layers', 'nb_bn_layers', 'nb_fc_layers']

train_dataset = select_train(list_var, train_dataset)
test_dataset1 = select_test(list_var, test_dataset1)
test_dataset2 = select_test(list_var, test_dataset2)
test_dataset3 = select_test(list_var, test_dataset3)

In [5]:
train_dataset.tail()

,Model_name,input_shape,ACET,input_size,nb_params_conv,flops,nb_layers,sum_activations,weighted_sum_neurons
1525,basic_model_64_15,"(1, 64, 64, 3)",185.188635,12288,37300460,10696805810,27,3050376,5309448448
1526,basic_model_32_44,"(1, 32, 32, 3)",182.426993,3072,32517012,23311729169,74,5247440,11618736064
1527,basic_model_112_17,"(1, 112, 112, 3)",167.433424,37632,15046344,51190739064,33,25374078,25571214384
1528,basic_model_75_37,"(1, 75, 75, 3)",159.081569,16875,18939872,26945702640,64,9258839,13449790920
1529,basic_model_32_23,"(1, 32, 32, 3)",135.091388,3072,16689184,34211079000,41,6237204,17086005248


In [6]:
Y = train_dataset.pop('ACET')
X = train_dataset

In [7]:
Y_e1 = test_dataset1.pop('ACET')
X_e1 = test_dataset1

Y_e2 = test_dataset2.pop('ACET')
X_e2 = test_dataset2

Y_e3 = test_dataset3.pop('ACET')
X_e3 = test_dataset3

In [8]:
X_train_model_names = X.pop('Model_name')
X_train_input_shapes = X.pop('input_shape')

In [9]:
X_test_model_names1 = X_e1.pop('Model_name')
X_test_input_shapes1 = X_e1.pop('input_shape')

X_test_model_names2 = X_e2.pop('Model_name')
X_test_input_shapes2 = X_e2.pop('input_shape')

X_test_model_names3 = X_e3.pop('Model_name')
X_test_input_shapes3 = X_e3.pop('input_shape')

In [10]:
y_test1 = Y_e1.values
y_test2 = Y_e2.values
y_test3 = Y_e3.values

In [11]:
polyn = PolynomialFeatures(degree=3)
xd = polyn.fit_transform(X.values)
clf_Ridge = linear_model.Ridge(alpha=0.0001, normalize=True,max_iter=1e5)
A = clf_Ridge.fit(xd,Y.values)

In [12]:
y_pred3 = clf_Ridge.predict(polyn.fit_transform(X_e3.values))
y_pred2 = clf_Ridge.predict(polyn.fit_transform(X_e2.values))
y_pred1 = clf_Ridge.predict(polyn.fit_transform(X_e1.values))

In [25]:
y_test = y_test3
y_pred = y_pred3
test_models = X_test_model_names3.values
test_input_shapes = X_test_input_shapes3.values

In [26]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9790201046175429
R2_value :  0.9777932376500473
Mean absolute error MAE : 73.88455962419762
Root Mean squared error RMSE : 119.37355222420038
Median absolute error MAD: 39.111735206751206


In [27]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE: ', round(max(mape), 2), '%.')
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE:  40.23 %.
MAPE: 13.19 %.
Accuracy: 86.81 %.


In [28]:
import csv
with open('./measured_vs_predicted_Poly_dim3.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])